# PERDIDO Geoparser


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/ludovicmoncla/perdido/blob/main/notebooks/demo_Geoparser.ipynb)





In [8]:
!pip install -i https://test.pypi.org/simple/ perdido==0.0.3
#!pip install perdido

Looking in indexes: https://test.pypi.org/simple/
  Attempting uninstall: perdido
    Found existing installation: perdido 0.0.2
    Uninstalling perdido-0.0.2:
      Successfully uninstalled perdido-0.0.2


In [21]:
from perdido import geoparser

In [22]:
p = geoparser.Geoparser()
doc = p.parse('Je visite la ville de Pau.')

In [23]:
doc._tei



'"<TEI>\\n   <teiheader/>\\n   <text>\\n      <body>\\n         <s>\\n            <w lemma=\\"je\\" type=\\"PRO\\" subtype=\\"PpvIL\\" id=\\"w0\\">Je</w>\\n            <phr type=\\"motion\\">\\n               <motionmedian>\\n                  <w lemma=\\"visiter\\" type=\\"V\\" id=\\"w1\\">visite</w>\\n               </motionmedian>\\n               <rs type=\\"ene\\" id=\\"en.0\\">\\n                  <rs type=\\"place\\" subtype=\\"ene\\" id=\\"en.1\\" start=\\"10\\" end=\\"12\\" startT=\\"2\\" endT=\\"6\\">\\n                     <term type=\\"place\\" start=\\"10\\" end=\\"12\\" startT=\\"2\\" endT=\\"4\\">\\n                        <w lemma=\\"le\\" type=\\"DET\\" subtype=\\"ART\\" id=\\"w2\\">la</w>\\n                        <w lemma=\\"ville\\" type=\\"N\\" id=\\"w3\\">ville</w>\\n                     </term>\\n                     <w lemma=\\"de\\" type=\\"PREP\\" id=\\"w4\\">de</w>\\n                     <rs type=\\"unknown\\" subtype=\\"no\\" id=\\"en.2\\" start=\\"22\\" end

In [24]:
doc._geojson

'{"type": "FeatureCollection", "features": "{\\"type\\": \\"FeatureCollection\\", \\"features\\": [{\\"type\\": \\"Feature\\", \\"geometry\\": {\\"type\\": \\"Point\\", \\"coordinates\\": [-0.369754, 43.29147]}, \\"properties\\": {\\"id\\": \\"en.1\\", \\"name\\": \\"ville de Pau\\", \\"sourceName\\": \\"Promenade, Pau, 64000, France\\", \\"type\\": \\"\\", \\"country\\": \\"France\\", \\"source\\": \\"osm\\"}}]}"}'